<a href="https://colab.research.google.com/github/Keizerbub/weather/blob/main/wrangling_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import framework

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=bcbc00fcd6f80cbae7f08cae6e42972827419b8fa239654b846db187d75e9102
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pandas as pd
import os
import gzip
import shutil
from pyspark.sql import SparkSession

#ouverture des fichiers

In [3]:
"""ouverture des fichiers"""
class wrangling:
    def __init__(self):
        self.spark = SparkSession.builder \
            .appName("HandlingFile") \
            .getOrCreate()



    def open_file(self, file_path):
        try:
            # Read the file into a DataFrame
            self.df = self.spark.read.csv(file_path, header=True, inferSchema=True, sep=";")  # Example for reading a CSV file
            # If it's a different type of file, use the appropriate method like `spark.read.json()`, etc.

            # Show the DataFrame content
            self.df.show()

        except Exception as e:
            print("Error occurred:", e)



    def close_spark(self):
        # Stop the SparkSession
        self.spark.stop()



    def select_columns(self, *columns, columns_state=True):
        if columns_state==False:
          columns='LAT','LON','AAAAMMJJHH','RR1','FF','TN50'

        try:
            # Select only the desired columns
            self.df = self.df.select(*columns)

            # Show the DataFrame content after selecting columns
            print("DataFrame after selecting columns:")
            self.df.show()

        except Exception as e:
            print("Error occurred:", e)



    def check_column_null(self, column_name):
        try:
            # Check if the column has any null values
            null_count = self.df.filter(self.df[column_name].isNull()).count()

            # Calculate the percentage of null values
            total_rows = self.df.count()
            self.null_percentage = (null_count / total_rows) * 100


            if null_count > 0:
                print(f"Column '{column_name}' has {null_count} null values for {self.null_percentage}.\n")
            else:
                print(f"Column '{column_name}' has no null values.\n")


        except Exception as e:
            print("Error occurred:", e)



    def check_all_columns_null(self):
      try:
          # Get all column names
          columns = self.df.columns

          # Apply check_column_null to each column
          for col in columns:
              self.check_column_null(col)
              #removing the missing col
              if self.null_percentage==100.0:
                  self.df = self.df.drop(col)
                  print(f'the column {col} was delete')

      except Exception as e:
          print("Error occurred:", e)

In [ ]:
folder=""
data=wrangling()
final_departement_folder = os.path.join(folder, 'final_departement')

# Liste des fichiers zip dans le dossier
files = [f for f in os.listdir(final_departement_folder) if f.endswith('.csv')]



# Dictionnaire pour stocker les DataFrames par préfixe de nom de fichier
file_dict = {}

# Parcourir chaque fichier zip
for fichier in files:
    # Chemin complet du fichier zip
    file_path_ = os.path.join(final_departement_folder, fichier)

    # Extraire le contenu du fichier zip dans le dossier de sortie
    data.open_file(file_path=file_path_)
    data.select_columns(columns_state=False)
    data.check_all_columns_null()
    # résultat
    print(f'file: {files} effectué\n')

Column 'LAT' has no null values.

Column 'LON' has no null values.

Column 'AAAAMMJJHH' has no null values.

Column 'RR1' has 84239 null values for 91.67374034171291.

Column 'FF' has 22060 null values for 24.006964849276308.

Column 'TN50' has 90170 null values for 98.12819675699205.

